In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import StratifiedKFold


In [ ]:
def collect_and_label(dataset): 
    

    df = pd.read_csv(dataset)
    alpha = df['scp_codes'].str.split("'").str[1].str[-2:]=='MI'  
    beta = df['scp_codes'].str.split("'").str[1]=='NORM'       
    df = df[alpha|beta]
    df['label'] = df['scp_codes'].str.split("'").str[1]      
      
    inst_c1 = df[df['label'] == 'NORM']
    inst_c1 = inst_c1.sample(n = 1000, random_state = 1)
    
    inst_c2 = df[df['label'] == 'ILMI']
    inst_c2 = inst_c2.sample(n = 393, random_state = 1)
    

    df_new = pd.concat([inst_c1, inst_c2], ignore_index = True)
    df_new = df_new.sample(frac = 1, random_state=42)
    
    return  df_new

In [ ]:
def div_and_label(dataframe):  
    
    X = dataframe[['ecg_id', 'filename_hr']]   
    X = X.to_numpy()
    y = dataframe['label']
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_y = encoder.transform(y)
    
    return (X, encoded_y)

In [ ]:
alpha = collect_and_label("C:/Users/Pushpam/Downloads/ptbxl_database.csv")  
gamma0, gamma1 = div_and_label(alpha)
gamma0 = gamma0[:,1]

print(gamma0.shape)
print(gamma1.shape)

In [ ]:
print(gamma0)

In [ ]:
gamma1

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
from scipy.signal import butter, filtfilt, detrend

directory = 'D:/Internship/MIDataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1'

X = []
for itr in range(gamma0.shape[0]):
    record_name = str(gamma0[itr])
    
    signal, meta_val = wfdb.rdsamp(directory + '/' + record_name)
    value = signal.T
    ecg_signals = value

    X.append(detrend(ecg_signals))
    

In [ ]:
X = np.array(X)
X.shape

In [ ]:
y_data = gamma1;
print(y_data.shape)

In [ ]:
# windowing X in xnew
# xnew = np.zeros((5*(X.shape[0]),12,1000))
xold = X
yold = y_data
xnew = []
ynew = []
patient_ids = []
for i in range(X.shape[0]):
    xnew.append(X[i,:,0:1000])
    xnew.append(X[i,:,1000:2000])
    xnew.append(X[i,:,2000:3000])
    xnew.append(X[i,:,3000:4000])
    xnew.append(X[i,:,4000:5000])
    for j in range(5):
        ynew.append(y_data[i])
        patient_ids.append(i)
xnew = np.array(xnew)
y_data = np.array(ynew)

In [ ]:
from scipy.signal import coherence
from glob import glob
import scipy.io as sio
import scipy.signal as sig
from scipy.signal import coherence, hilbert 

cross_corr_matrices_list = []
coherence_matrices_list = []
pli_matrices_list = []
plv_matrices_list = []

for patient_data in xnew:
    cross_corr_matrix = np.zeros((12, 12))
    coherence_matrix = np.zeros((12, 12))
    pli_matrix = np.zeros((12, 12))
    plv_matrix = np.zeros((12, 12))

    
    for i in range(12):
        for j in range(i, 12):
            lead_i = patient_data[i]
            lead_j = patient_data[j]
            
            cross_corr = np.corrcoef(lead_i, lead_j)[0, 1]
            
            f, coh = coherence(lead_i, lead_j)  
            coherence_value = np.mean(coh)  # Storing the average coherence value
            
            
            analytic_i = hilbert(lead_i)
            analytic_j = hilbert(lead_j)
            
            phase_i = np.angle(analytic_i)
            phase_j = np.angle(analytic_j)
                    
            phase_diff = phase_i - phase_j
              
            pli = np.abs(np.mean(np.sign(np.sin(phase_diff))))
               
            plv = np.abs(np.mean(np.exp(1j * phase_diff)))

            cross_corr_matrix[i, j] = cross_corr
            cross_corr_matrix[j, i] = cross_corr
        
            coherence_matrix[i, j] = coherence_value
            coherence_matrix[j, i] = coherence_value    
            
            pli_matrix[i, j] = pli
            pli_matrix[j, i] = pli
            
            plv_matrix[i, j] = plv
            plv_matrix[j, i] = plv
    
    
    cross_corr_matrices_list.append(cross_corr_matrix)  
    coherence_matrices_list.append(coherence_matrix)
    pli_matrices_list.append(pli_matrix)
    plv_matrices_list.append(plv_matrix)
    
def create_K(A_i, B_i, C_i, D_i):
    top_row = np.concatenate((A_i, B_i), axis=1)
    bottom_row = np.concatenate((C_i, D_i), axis=1)
    return np.concatenate((top_row, bottom_row), axis=0)

K_matrices=[]
for i in range(xnew.shape[0]):
    K_i = create_K(cross_corr_matrices_list[i], coherence_matrices_list[i], pli_matrices_list[i], plv_matrices_list[i])
    K_matrices.append(K_i)

In [13]:
# To load the matrices back in another script, you can use the following code:
loaded_data = np.load('matrices_data.npz')
cross_corr_loaded = loaded_data['cross_corr']
coherence_loaded = loaded_data['coherence']
pli_loaded = loaded_data['pli']
plv_loaded = loaded_data['plv']

ynew = np.load(r"ynew_ALMI_800+164_kernels5000.npy")
patient_ids = np.load(r"patient_ids_ILMI_1000+393.npy")

In [ ]:
# t = np.array(K_matrices).reshape(X.shape[0],12,12,4)
# print(t.shape)

In [15]:
t = []
for i in range(ynew.shape[0]):
    t2 = []
    t2.append(cross_corr_loaded[i])
    t2.append(pli_loaded[i])
    t2.append(plv_loaded[i])
    t2.append(coherence_loaded[i])
    t.append(np.array(t2).reshape(12,12,4))
t = np.array(t)

In [17]:
t.shape

(6965, 12, 12, 4)

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale

In [9]:
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D

In [10]:
patient_ids = np.array(patient_ids)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

# Assuming t is a list of 4 matrices, each of shape (12, 12)
# Combine the 4 matrices into a single 4-channel input
X_data = np.stack(t, axis=-1)

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, ynew, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    tf.keras.backend.clear_session()

    model = Sequential()

    # Use 4 input channels
    model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 4)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=BinaryCrossentropy(),
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=16,
                        epochs=20,
                        verbose=1,
                        validation_split=0.25)

    predictions = model.predict(X_test)
    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


Epoch 1/20
262/262 [==============================] - 1s 2ms/step - loss: 0.2977 - accuracy: 0.8702 - val_loss: 0.3811 - val_accuracy: 0.8478
Epoch 2/20
262/262 [==============================] - 0s 1ms/step - loss: 0.2174 - accuracy: 0.9124 - val_loss: 0.2560 - val_accuracy: 0.8909
Epoch 3/20
262/262 [==============================] - 0s 1ms/step - loss: 0.1885 - accuracy: 0.9253 - val_loss: 0.2592 - val_accuracy: 0.9031
Epoch 4/20
262/262 [==============================] - 0s 1ms/step - loss: 0.1633 - accuracy: 0.9375 - val_loss: 0.2147 - val_accuracy: 0.9088
Epoch 5/20
262/262 [==============================] - 0s 1ms/step - loss: 0.1448 - accuracy: 0.9435 - val_loss: 0.2883 - val_accuracy: 0.8995
Epoch 6/20
262/262 [==============================] - 0s 1ms/step - loss: 0.1226 - accuracy: 0.9552 - val_loss: 0.2230 - val_accuracy: 0.9153
Epoch 7/20
262/262 [==============================] - 0s 1ms/step - loss: 0.1063 - accuracy: 0.9605 - val_loss: 0.2386 - val_accuracy: 0.9167
Epoch 

262/262 [==============================] - 0s 1ms/step - loss: 0.0313 - accuracy: 0.9892 - val_loss: 0.4193 - val_accuracy: 0.9031
Epoch 17/20
262/262 [==============================] - 0s 1ms/step - loss: 0.0231 - accuracy: 0.9928 - val_loss: 0.4182 - val_accuracy: 0.9081
Epoch 18/20
262/262 [==============================] - 0s 1ms/step - loss: 0.0191 - accuracy: 0.9928 - val_loss: 0.4627 - val_accuracy: 0.9060
Epoch 19/20
262/262 [==============================] - 0s 1ms/step - loss: 0.0140 - accuracy: 0.9957 - val_loss: 0.5099 - val_accuracy: 0.9009
Epoch 20/20
44/44 [==============================] - 0s 626us/step
[[ 76   9]
 [  8 186]]
Accuracy: 0.9390681003584229
Sensitivity: 0.9587628865979382
Specificity: 0.8941176470588236
Epoch 1/20
262/262 [==============================] - 1s 2ms/step - loss: 0.3016 - accuracy: 0.8742 - val_loss: 0.2500 - val_accuracy: 0.9046
Epoch 2/20
262/262 [==============================] - 0s 1ms/step - loss: 0.2337 - accuracy: 0.9007 - val_loss: 0.2